# sys.argv

参考资料：
- [sys.argv是什么？](https://www.zhihu.com/question/23711222)
- [Python：sys.argv[]用法](http://www.cnblogs.com/cython/articles/2196715.html)
- [Python中 sys.argv[]的用法简明解释](http://www.cnblogs.com/aland-1415/p/6613449.html)

`sys.argv[]` 是用来获取命令行参数的，`sys.argv[0]`表示代码本身文件路径，所以参数从 `1` 开始：

In [15]:
import sys

In [16]:
sys.argv[0]

'C:\\Anaconda3\\lib\\site-packages\\ipykernel_launcher.py'

```py
import sys


def readfile(filename):
    ''' Print a file to the standard output. '''
    f = file(filename)
    while True:
        line = f.readline()
        if len(line) == 0:
            break
        print(line)
    f.close()


# Script starts from here
if len(sys.argv) < 2:
    print(' NO action specified.')
    sys.exit()

if sys.argv[1].startswith('--'):
    option = sys.argv[1][2:]
    if option == 'version':
        print(' version 1.2 ')
    elif option == 'help':
        print('''This program prints files to the standard output. 
             Any number of files can be specified. 
             Options include: 
             --version : Prints the version number 
             --help     : Display this help''')
    else:
        print('Unknow option.')
else:
    for filename in sys.argv[1:]:
        readfile(filename)        
```

保存为 `sample.py`,我们验证一下：

In [17]:
!python sample.py --version

 version 1.2 


In [18]:
!python sample.py --help

This program prints files to the standard output. 
             Any number of files can be specified. 
             Options include: 
             --version : Prints the version number 
             --help     : Display this help


`sys.argv[]` 说白了就是一个从程序外部获取参数的桥梁，这个“外部”很关键，所以那些试图从代码来说明它作用的解释一直没看明白。因为我们从外部取得的参数可以是多个，所以获得的是一个列表（`list`)，也就是说 `sys.argv` 其实可以看作是一个列表，所以才能用 `[]` 提取其中的元素。其第一个元素是程序本身，随后才依次是外部给予的参数。（理由也许是：计算机内存可以看作为一个大的列表，第一的元素是程序本身，之后的才是从外部获取的信息。）

与 `sample.py` 同一目录下，新建 `a.txt` 的记事本文件，内容为：`test argv;`

In [90]:
!python sample.py a.txt  

test argv；


# python 模块之 `codecs`: 自然语言编码转换

python 对多国语言的处理是支持的很好的，它可以处理现在任意编码的字符，这里深入的研究一下 python 对多种不同语言的处理。

有一点需要清楚的是，当 python 要做编码转换的时候，会借助于内部的编码，转换过程是这样的：
     `原有编码 -> 内部编码 -> 目的编码`

python 的内部是使用 unicode 来处理的，但是 unicode 的使用需要考虑的是它的编码格式有两种，一是 `UCS-2`，它一共有 $65536$ 个码位，另一种是 `UCS-4`，它有 $2147483648$个码位。对于这两种格式，python 都是支持的，这个是在编译时通过 `--enable-unicode=ucs2` 或 `--enable-unicode=ucs4` 来指定的。

那么我们自己默认安装的 python 有的什么编码怎么来确定呢？有一个办法，就是通过 `sys.maxunicode` 的值来判断： 

In [30]:
import sys
print(sys.maxunicode)

1114111


如果输出的值为 `65535`,那么就是 `UCS-2`,如果输出是 `1114111` 就是 `UCS-4` 编码。

我们要认识到一点：当一个字符串转换为内部编码后，它就不是`str`类型了！

参考资料：
- [Python3如何解决字符编码问题详解](http://www.jb51.net/article/111864.htm)
- [Python3.5——字符编码转换（transcoding）详解](http://blog.csdn.net/loveliuzz/article/details/77799267)
- [Python3如何解决字符编码问题详解](https://www.oudahe.com/p/13645/)
- [python3字符串编码总结-str(unicode)_bytes](http://blog.csdn.net/Setul/article/details/52203894)

因为计算机只能处理数字，如果要处理文本，就必须先把文本转换为数字才能处理。最早的计算机在设计时采用$8$个比特（`bit`）作为一个字节（`byte`），所以，一个字节能表示的最大的整数就是$255$（二进制 `11111111`= 十进制 `255`），如果要表示更大的整数，就必须用更多的字节。比如两个字节可以表示的最大整数是 `65535`，$4$ 个字节可以表示的最大整数是 `4294967295`。

由于计算机是美国人发明的，因此，最早只有$127$个字母被编码到计算机里，也就是大小写英文字母、数字和一些符号，这个编码表被称为 `ASCII`编码，比如大写字母`A`的编码是`65`，小写字母`z`的编码是`122`。

## python3字符编码

Python3 最重要的一项改进之一就是解决了 Python2 中字符串与字符编码遗留下来的这个大坑。
Python2 字符串设计上的一些缺陷：
- 使用 ASCII 码作为默认编码方式，对中文处理很不友好。
- 把字符串的牵强地分为 unicode 和 str 两种类型，误导开发者

当然这并不算 Bug，只要处理的时候多留心也可以避免这些坑。但在 Python3 两个问题都很好的解决了。
首先，Python3 把系统默认编码设置为 UTF-8：

In [42]:
import sys
sys.getdefaultencoding()

'utf-8'

然后，文本字符和二进制数据区分得更清晰，分别用 `str` 和 `bytes` 表示。文本字符全部用 `str` 类型表示，`str` 能表示 `Unicode` 字符集中所有字符，而二进制字节数据用一种全新的数据类型，用 `bytes` 来表示。

### `str`

In [43]:
a = "a"
a

'a'

In [44]:
type(a)

str

In [45]:
b = "禅"
b

'禅'

In [46]:
type(b)

str

### `bytes`
Python3 中，在字符引号前加 `b`，明确表示这是一个 `bytes` 类型的对象，实际上它就是一组二进制字节序列组成的数据，`bytes` 类型可以是 `ASCII` 范围内的字符和其它十六进制形式的字符数据，但不能用中文等非 `ASCII` 字符表示。

In [47]:
c = b'a'
c

b'a'

In [48]:
type(c)

bytes

In [49]:
d = b'\xe7\xa6\x85'
d

b'\xe7\xa6\x85'

In [50]:
type(d)

bytes

In [51]:
e = b'禅'

SyntaxError: bytes can only contain ASCII literal characters. (<ipython-input-51-34662558ba55>, line 1)

`bytes` 类型提供的操作和 `str` 一样，支持分片、索引、基本数值运算等操作。但是 `str` 与 `bytes` 类型的数据不能执行 `+` 操作，尽管在`py2`中是可行的。

In [52]:
b"a"+b"c"

b'ac'

In [53]:
b"a"*2

b'aa'

In [54]:
b"abcdef\xd6"[1:]

b'bcdef\xd6'

In [55]:
b"abcdef\xd6"[-1]

214

In [56]:
b"a" + "b"

TypeError: can't concat str to bytes

### python2 与 python3 字节与字符的对应关系

python2|python3|表现|转换|作用
:-|:-|:-|:-|:-
`str`|`bytes`|字节|`encode`|存储
`unicode`|`str`|字符|`decode`|显示

`encode` 负责字符到字节的编码转换。默认使用 `UTF-8` 编码准换。

In [58]:
s = "Python之禅"
s.encode()

b'Python\xe4\xb9\x8b\xe7\xa6\x85'

In [59]:
s.encode("gbk")

b'Python\xd6\xae\xec\xf8'

`decode` 负责字节到字符的解码转换，通用使用 `UTF-8` 编码格式进行转换。

In [60]:
b'Python\xe4\xb9\x8b\xe7\xa6\x85'.decode()

'Python之禅'

In [62]:
b'Python\xd6\xae\xec\xf8'.decode("gbk")

'Python之禅'

### Unicode
Unicode 把所有语言都统一到一套编码里，这样就不会再有乱码问题了。

Unicode 标准也在不断发展，但最常用的是用两个字节表示一个字符（如果要用到非常偏僻的字符，就需要 $4$个字节）。现代操作系统和大多数编程语言都直接支持 Unicode。

ASCII 编码和 Unicode 编码的区别：ASCII 编码是 $1$ 个字节，而 Unicode 编码通常是 $2$ 个字节。
#### 示例
- 字母 `A` 用 ASCII 编码是十进制的 `65`，二进制的 `01000001`；
- 字符 `0` 用 ASCII 编码是十进制的 `48`，二进制的 `00110000`，注意字符 `'0'` 和整数 `0` 是不同的；
- 汉字已经超出了 ASCII 编码的范围，用 Unicode 编码是十进制的 `20013`，二进制的 `01001110 00101101`。

如果把 ASCII 编码的 `A` 用 Unicode 编码，只需要在前面补 `0`就可以，因此，`A` 的 Unicode 编码是 `00000000 01000001`。 

新的问题又出现了：如果统一成 Unicode 编码，乱码问题从此消失了。但是，如果你写的文本基本上全部是英文的话，用 Unicode 编码比 ASCII 编码需要多一倍的存储空间，在存储和传输上就十分不划算。

所以，又出现了把 Unicode 编码转化为“可变长编码”的 `UTF-8` 编码。UTF-8 编码把一个 Unicode 字符根据不同的数字大小编码成 $1-6$ 个字节，常用的英文字母被编码成 $1$ 个字节，汉字通常是 $3$ 个字节，只有很生僻的字符才会被编码成 $4-6$个字节。如果你要传输的文本包含大量英文字符，用 `UTF-8` 编码就能节省空间：

UTF-8 编码有一个额外的好处，就是 ASCII 编码实际上可以被看成是 UTF-8 编码的一部分，所以，大量只支持 ASCII 编码的历史遗留软件可以在UTF-8 编码下继续工作。

在计算机内存中，统一使用 Unicode 编码，当需要保存到硬盘或者需要传输的时候，就转换为 `UTF-8` 编码。

- 用记事本编辑的时候，从文件读取的 UTF-8字 符被转换为 Unicode 字符到内存里，编辑完成后，保存的时候再把 Unicode 转换为 UTF-8 保存到文件;
- 浏览网页的时候，服务器会把动态生成的 Unicode 内容转换为 UTF-8 再传输到浏览器，所以你看到很多网页的源码上会有类似·<meta charset="UTF-8" />·的信息，表示该网页正是用的 UTF-8 编码。


### Python 的字符串
在最新的 Python 3 版本中，字符串是以 Unicode 编码的，也就是说，Python 的字符串支持多语言，例如：

In [63]:
print('包含中文的str')

包含中文的str


对于单个字符的编码，Python 提供了`ord()`函数获取字符的整数表示，`chr()`函数把编码转换为对应的字符：

In [64]:
ord('A')

65

In [65]:
ord('中')

20013

In [66]:
chr(66)

'B'

In [67]:
chr(25991)

'文'

如果知道字符的整数编码，还可以用十六进制这么写 `str`：

In [69]:
'\u4e2d\u6587' 

'中文'

#### `byte`
由于 Python 的字符串类型是 `str`，在内存中以 Unicode 表示，一个字符对应若干个字节。如果要在网络上传输，或者保存到磁盘上，就需要把 `str` 变为以字节为单位的 `bytes`。
Python 对 bytes 类型的数据用带 `b` 前缀的单引号或双引号表示：

In [72]:
x = b'ABC'
x

b'ABC'

纯英文的 `str` 可以用 ASCII 编码为 `bytes`，内容是一样的，含有中文的 `str` 可以用 UTF-8 编码为 bytes。含有中文的 `str` 无法用 ASCII 编码，因为中文编码的范围超过了 ASCII 编码的范围，Python 会报错。

在 `bytes` 中，无法显示为 ASCII 字符的字节，用`\x##`显示。

In [73]:
b'ABC'.decode('ascii')

'ABC'

In [74]:
b'\xe4\xb8\xad\xe6\x96\x87'.decode('utf-8')

'中文'

要计算 `str` 包含多少个字符，可以用`len()`函数

In [75]:
len('ABC')

3

In [76]:
len('中文')

2

`len()` 函数计算的是 `str` 的字符数，如果换成 `bytes`，`len()`函数就计算字节数：

In [78]:
len(b'\xe4\xb8\xad\xe6\x96\x87')

6

In [80]:
len(b'ABC')

3

In [81]:
len('中文'.encode('utf-8'))

6

$1$ 个中文字符经过 UTF-8 编码后通常会占用 $3$ 个字节，而 $1$ 个英文字符只占用 $1$ 个字节。

在操作字符串时，我们经常遇到 `str` 和 `bytes` 的互相转换。为了避免乱码问题，应当始终坚持使用 `UTF-8` 编码对 `str` 和 `bytes` 进行转换。

Python 源代码也是一个文本文件，所以，当你的源代码中包含中文的时候，在保存源代码时，就需要务必指定保存为 UTF-8 编码。当 Python 解释器读取源代码时，为了让它按 UTF-8 编码读取，我们通常在文件开头写上这两行:
```py
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
```

第二行注释是为了告诉 Python 解释器，按照 UTF-8 编码读取源代码，否则，你在源代码中写的中文输出可能会有乱码。

##### [字符编码和 python 使用 `encode`,`decode` 转换 `utf-8`, `gbk`, `gb2312`](http://www.cnblogs.com/jxzheng/p/5186490.html)

### ASCII 码
标准 ASCII 码使用 $7$ 位二进制数表示大写或小写字母，数字 $0$ 到 $9$ 标点符号以及在美式英语中使用的特殊控制字符。

在标准ASCII码中，最高位（`b7`）用作奇偶校验位，所谓**奇偶校验**，是指在代码传送过程中用来检验是否出现错误的一种方法，一般分奇校验和偶校验两种。
- 奇校验规定：正确的代码一个字节中 `1` 的个数必须是奇数，若非奇数，则在最高位 `b7` 添 `1`；
- 偶校验规定：正确的代码一个字节中 `1` 的个数必须是偶数，若非偶数，则在最高位 `b7`添 `1`。

后 $128$ 个称为*扩展 ASCII 码*。许多基于 x86 的系统都支持使用扩展（或“高”）ASCII。扩展 ASCII 码允许将每个字符的第 $8$ 位用于确定附加的 $128$ 个特殊符号字符、外来语字母和图形符号。

常见的ASCII码大小：
- 换行 `LF`为 `0x0A`，回车`CR`为`0x0D`，空格为`0x20`，`0'`为`0x30`，`'A'`为`0x41`，`'a'`为`0x61`

查询 ASCII 技巧，方便查询 ASCII 码对应的字符：新建一个文本文档，按住 `ALT` + 要查询的码值（注意，这里是十进制），松开即可显示出对应字符。例如：按住`ALT` + `97`,则会显示出 `'a'`。

#### 扩展ASCII码
扩展 ASCII 码是从 $128-255$ 的字符。

#### Unicode 编码
注意：Unicode 只是一个符号集，它规定了符号的二进制代码，却没有规定二进制代码如何存储。
所称的 Unicode 编码指的是 UCS 编码方式，即直接存入符号的 Unicode 二进制代码。
#### UTF-8 编码
UTF-8 是互联网上使用最广的一种 Unicode 的实现方式。

UTF-8 是一种变长的编码方式，它使用 $1-4$ 个字节表示一个符号，根据不同的符号选择不同长度的字节表示。

UTF-8 的编码规则很简单，只有二条：
1. 对于单字节的符号，字节的第一位设为0，后面7位为这个符号的unicode码。因此对于英语字母，UTF-8 编码和 ASCII 码是相同的。
2. 对于 $n$ 字节的符号（$n>1$），第一个字节的前 $n$ 位都设为 `1`，第 $n+1$ 位设为 `0`，后面字节的前两位一律设为 `10`。剩下的没有提及的二进制位，全部为这个符号的 unicode 码。

## 解决 python 乱码问题
 
字符串在 python 内部中是采用 unicode 的编码方式，所以其他语言先 decode 转换成 unicode 编码，再 encode 转换成utf8编码。编码是一种用二进制数据表示抽象字符的方式，utf8 是一种编码方式。

代码中的字符串编码默认和代码文件编码相同。

python2 中的 unicode 和 python3 中的 `str` 等价。可以查看 `s.__class__`，如果为`<class 'str'>`则为 unicode 编码及文本数据，如果为`<class 'bytes'>` 则为 utf8 编码及二进制数据。`str(s, 'utf8')` 和 `s.decode('utf8')` 等价。

如果字符串在代码中被定义为 `s=u'中文'`，则 `s` 就是 python 内部编码 `unicode`。
unicode类型再解码会报错。

判断一个字符串是否为 unicode 方法 `isinstance(s, unicode)`，python2 中的 `unicode` 和 python3 中的 `str` 等价，所以在 python3 中判断一个字符串是否为 unicode 方法为 `isinstance(s, str)`。

获取系统默认编码：

In [84]:
import sys
print(sys.getdefaultencoding())

utf-8


In [85]:
# coding=utf-8

s='中文'


if(isinstance(s, str)):
# s 为 u'中文'
    s.encode('gb2312')
else:
# s为 '中文'
    s.decode('utf8').encode('gb2312')

### 采用标准库 ``codecs``模块
`codecs.open(filename, mode='r', encoding=None, errors='strict', buffering=1)`

使用上边这种方式读进来 utf-8 文件，会自动转换为 unicode。但必须明确该文件类型为 utf8 类型。如果是文件中有汉字，不是一个字节一个字节地读而是整个汉字的所有字节读进来然后转换成 unicode（猜想跟汉字的utf8编码有关）。

In [93]:
def readfile(filename):
    ''' Print a file to the standard output. '''
    f = codecs.open(filename, encoding='utf-8')
    while True:
        line = f.readline()
        if len(line) == 0:
            break
        print(line)
    f.close()

readfile('a.txt')

test argv；


下边的代码也是一种使用 `codecs`的读写方式：

```py
# coding=utf-8
import codecs

fin = open("a.txt", 'r')
fout = open("utf8.txt", 'w')

reader = codecs.getreader('gbk')(fin)
writer = codecs.getwriter('gbk')(fout)

data = reader.read(10)
# 10 是最大字节数，默认值为-1表示尽可能大。可以避免一次处理大量数据
while data:
    writer.write(data)
    data = reader.read(10)
```